In [1]:
from google.colab import files
from IPython.display import Image
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
cd '/gdrive/My Drive/Colab Notebooks/team_UBUNTU'

/gdrive/My Drive/Colab Notebooks/team_UBUNTU


In [3]:
pip install -U sentence-transformers

     |████████████████████████████████| 71kB 2.3MB/s 
     |████████████████████████████████| 1.1MB 9.6MB/s 
     |████████████████████████████████| 1.1MB 24.4MB/s 
     |████████████████████████████████| 3.0MB 31.9MB/s 
     |████████████████████████████████| 890kB 41.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.8-cp36-none-any.whl size=101996 sha256=20c4af89390ed063641bf374216ad606594932db70f6aa2fd06104fcb93a48a0
  Stored in directory: /root/.cache/pip/wheels/27/ec/b3/d12cc8e4daf77846db6543033d3a5642f204c0320b15945647
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=41f88395fd41b8791f37c018d07224af8af1aa03417d2bc7c28520dc4a2d4727
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [4]:
from sentence_transformers import SentenceTransformer
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix
%matplotlib inline
import nltk
import string
import pandas as pd
import numpy as np
import re
from textblob import TextBlob
import string
import torch
import pickle
from bs4 import BeautifulSoup
from gensim.summarization.summarizer import summarize
nltk.download('averaged_perceptron_tagger')
from sentence_transformers import SentenceTransformer
nltk.download('punkt')
nltk.download('wordnet')
import torch
import json 

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# ACCESING THE DATA

In [13]:
import chardet
with open('disease_components.csv', 'rb') as f:
    result = chardet.detect(f.read())

In [ ]:
result['encoding']

In [14]:
df = pd.read_csv('disease_components.csv',encoding=result['encoding'])
df.dropna(subset = ["Symptoms"], inplace=True)
df.rename(columns={'*':'disease'},inplace=True)
df = df.fillna("")
df = df.reset_index()
df.drop('index',axis=1,inplace=True)

In [ ]:
dis_symp = []
for i in range(len(df)):
  dis_symp.append(df['Symptoms'][i])

In [10]:
def pre_processing(question):
    def lemmatize_with_pos_tag(sentence):
        tokenized_sentence = TextBlob(sentence)
        tag_dict = {"J": 'a', "N": 'n', "V": 'v', "R": 'r'}
        words_and_tags = [(word, tag_dict.get(pos[0], 'n')) for word, pos in tokenized_sentence.tags]
        lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
        return " ".join(lemmatized_list)
    question = BeautifulSoup(question, 'html.parser').get_text()
    question = question.lower()
    question = question.replace('[','')
    question = question.replace(']','')
    question.translate(str.maketrans(" ", " ", string.punctuation))
    question = lemmatize_with_pos_tag(question)
    return question

In [ ]:
symp = []
for i in range(len(dis_symp)):
  symp.append(pre_processing(dis_symp[i]))

In [ ]:
symp[145]

In [ ]:
with open('symp.pkl', 'wb') as f:
  pickle.dump(symp, f)

In [6]:
with open('symp.pkl', 'rb') as f:
  symps = pickle.load(f)

In [ ]:
symps

In [7]:
model = SentenceTransformer('model')

In [8]:
sentence_embeddings = model.encode(symps)

In [16]:
question = 'I am suffering from sleeplessness ,I can sleep at day but not during nighttime'
queries = pre_processing(question)
print(queries)
query_embeddings = model.encode([queries])

i be suffer from sleeplessness i can sleep at day but not during nighttime


In [17]:
import scipy
number_top_matches = 10
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]
    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", question)
    print("\nTop 10 most matching disease u might have ")

    for idx, distance in results[0:number_top_matches]:
        print(df['disease'][idx],idx, "(Cosine Score: %.4f)" % (1-distance))





Query: I am suffering from sleeplessness ,I can sleep at day but not during nighttime

Top 10 most matching disease u might have 
Sleep disorders 827 (Cosine Score: 0.6841)
Somnambulism (See: Somnambulism, also known asSleepwalking) 845 (Cosine Score: 0.6343)
Sleepiness, daytime (See: Sleepiness, daytime, also known asNarcolepsy) 829 (Cosine Score: 0.6231)
Sleep terrors (night terrors) 828 (Cosine Score: 0.6121)
Sleeplessness (See: Sleeplessness, also known asInsomnia) 830 (Cosine Score: 0.5952)
Sleep apnea 823 (Cosine Score: 0.5797)
Snoring 838 (Cosine Score: 0.5749)
Sleep apnea, central 825 (Cosine Score: 0.5730)
Nightmare disorder 538 (Cosine Score: 0.5692)
Jet lag disorder 390 (Cosine Score: 0.5070)


In [51]:
df['Symptoms'][1028]

'[\'Normal urine color varies, depending on how much water you drink. Fluids dilute the yellow pigments in urine, so the more you drink, the clearer your urine looks. When you drink less, the color becomes more concentrated. Severe dehydration can produce urine the color of amber.\', "But urine can turn colors far beyond what\'s normal, including red, blue, green, dark brown and cloudy white.", \'Seek medical attention if you have:\', \'Visible blood in your urine. Bloody urine is common in urinary tract infections and kidney stones. These problems usually cause pain. Painless bleeding might signal a more-serious problem, such as cancer.\', \'Dark or orange urine. If your urine is dark or orange — particularly if you also have pale stools and yellow skin and eyes — your liver might be malfunctioning.\']'

In [18]:
df

,disease,link,Symptoms,Overview,Causes,Risk factors,diagnosis,treatment,remedies,medication
0,Acanthosis nigricans,https://www.mayoclinic.org/diseases-conditions...,"[""Skin changes are the only signs of acanthosi...","['', 'Acanthosis nigricans is a skin condition...",['Acanthosis nigricans has been associated wit...,"['Acanthosis nigricans risk factors include:',...",['Acanthosis nigricans is typically detected d...,"['In many situations, treating the underlying ...",,
1,Achalasia,https://www.mayoclinic.org/diseases-conditions...,['Achalasia symptoms generally appear graduall...,['Achalasia is a rare disorder that makes it d...,['The exact cause of achalasia is poorly under...,,['Achalasia can be overlooked or misdiagnosed ...,['Achalasia treatment focuses on relaxing or s...,,
2,Achilles tendon rupture,https://www.mayoclinic.org/diseases-conditions...,"[""Although it's possible to have no signs or s...","['', 'The Achilles tendon is a strong fibrous ...",['Your Achilles tendon helps you point your fo...,['Factors that may increase your risk of Achil...,"['During the physical exam, your doctor will i...",['Treatment for a ruptured Achilles tendon oft...,,
3,Acute coronary syndrome,https://www.mayoclinic.org/diseases-conditions...,['The signs and symptoms of acute coronary syn...,['Acute coronary syndrome is a term used to de...,['Acute coronary syndrome usually results from...,['The risk factors for acute coronary syndrome...,['If you have signs or symptoms associated wit...,['The immediate goals of treatment for acute c...,['Heart healthy lifestyle changes are an impor...,"['Depending on your diagnosis, medications for..."
4,Adenomyosis,https://www.mayoclinic.org/diseases-conditions...,"['Sometimes, adenomyosis causes no signs or sy...","['', 'With adenomyosis, the same tissue that l...","[""The cause of adenomyosis isn't known. There ...","['Risk factors for adenomyosis include:', 'Mos...",['Some other uterine conditions can cause sign...,"['Adenomyosis often goes away after menopause,...",[' To ease pelvic pain and cramping related to...,
...,...,...,...,...,...,...,...,...,...,...
1101,Yips,https://www.mayoclinic.org/diseases-conditions...,['The most common symptom associated with the ...,['The yips are involuntary wrist spasms that o...,"[""In some people, the yips are a type of focal...","['The yips tend to be associated with:', 'Olde...",,['Because the yips may be related to overuse o...,,
1102,Zika virus,https://www.mayoclinic.org/diseases-conditions...,['As many as 4 out of 5 people infected with t...,['The mosquito that carries the Zika virus is ...,['The Zika virus is transmitted primarily thro...,['Factors that put you at greater risk of deve...,['The mosquito that carries the Zika virus is ...,['No specific antiviral treatment for the Zika...,,
1103,Zollinger-Ellison syndrome,https://www.mayoclinic.org/diseases-conditions...,['Signs and symptoms of Zollinger-Ellison synd...,['Zollinger-Ellison syndrome is a rare conditi...,"['', 'The pancreas is a large organ that lies ...","[""If you have a blood relative, such as a sibl...",['Your doctor will base a diagnosis on the fol...,['Treatment of Zollinger-Ellison syndrome addr...,,
1104,"Zoster (See: Zoster, also known asShingles)",https://www.mayoclinic.org/diseases-conditions...,"['', 'Shingles is characterized by pain or a t...",['Shingles is a viral infection that causes a ...,"['', 'The shingles rash is associated with an ...",['Anyone who has ever had chickenpox can devel...,['Shingles is usually diagnosed based on the h...,"[""There's no cure for shingles, but prompt tre...","['Taking a cool bath or using cool, wet compre...",
